# Agent Engine の A2A 対応機能の利用例 (2)

Agent Engine に A2A サーバーの機能がデプロイできるようになりました。

たとえば、下記の記事では、Agent Engine にデプロイした ADK Agent の前段に Cloud Run を用いて A2A サーバーを用意していましたが、Agent Engine に A2A サーバーの機能も同時にデプロイできるようになります。

- [A2A リモートエージェント対応のマルチエージェントシステム](https://zenn.dev/google_cloud_jp/articles/d24a22a87a84f8)

このノートブックでは、上記の記事で Cloud Run にデプロイしていた A2A サーバーの機能を Agent Engine に移行する例を紹介します。

この例では、ADK が提供する A2A 対応モジュールを利用して、ADK の標準的なコードで実装できるようにしてあります。

## パッケージインストール

In [ ]:
%pip install --user \
    google-adk==1.14.1 \
    google-genai==1.36.0 \
    google-cloud-aiplatform==1.113.0 \
    a2a-sdk==0.3.5

カーネルを再起動します。

In [3]:
import IPython
app = IPython.Application.instance()
_ = app.kernel.do_shutdown(True)

## 事前準備

In [2]:
import copy, json, os, uuid
import vertexai
from vertexai import agent_engines

from google.genai.types import Part, Content, HttpOptions
from google.adk.agents.llm_agent import LlmAgent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.events import Event
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner

from google.adk.agents.callback_context import CallbackContext
from google.adk.models import LlmResponse, LlmRequest
from google.adk.agents.sequential_agent import SequentialAgent

[PROJECT_ID] = !gcloud config list --format 'value(core.project)'
[PROJECT_NUMBER] = !gcloud projects describe {PROJECT_ID} --format='value(projectNumber)'
LOCATION = 'us-central1'

vertexai.init(project=PROJECT_ID, location=LOCATION,
              staging_bucket=f'gs://{PROJECT_ID}')

os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'True'

root agent をローカルで実行する簡易アプリのクラスを定義します。

In [3]:
class LocalApp:
    def __init__(self, agent, app_name='default_app', user_id='default_user'):
        self._agent = agent
        self._app_name = app_name
        self._user_id = user_id
        self._runner = Runner(
            app_name=self._app_name,
            agent=self._agent,
            artifact_service=InMemoryArtifactService(),
            session_service=InMemorySessionService(),
            memory_service=InMemoryMemoryService(),
        )
        self._session = None
        
    async def stream(self, query):
        if not self._session:
            self._session = await self._runner.session_service.create_session(
                app_name=self._app_name,
                user_id=self._user_id,
                session_id=uuid.uuid4().hex,
            )
        content = Content(role='user', parts=[Part(text=query)])
        async_events = self._runner.run_async(
            user_id=self._user_id,
            session_id=self._session.id,
            new_message=content,
        )
        result = []
        async for event in async_events:
            if DEBUG:
                print(f'====\n{event}\n====')
            if (event.content and event.content.parts):
                response = '\n'.join([p.text for p in event.content.parts if p.text])
                if response:
                    print(response)
                    result.append(response)
        return result

## リモートエージェントの定義

In [4]:
instruction = '''
あなたの役割は、記事の執筆に必要な情報を収集して調査レポートにまとめる事です。
指定されたテーマの記事を執筆する際に参考となるトピックを５項目程度のリストにまとめます。
後段のエージェントがこのリストに基づいて、調査レポートを作成します。

* 出力形式
日本語で出力。
'''

research_agent1 = LlmAgent(
    name='research_agent1',
    model='gemini-2.5-flash',
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント（テーマ選定）',
    instruction=instruction,
)

In [5]:
instruction = '''
あなたの役割は、記事の執筆に必要な情報を収集して調査レポートにまとめる事です。
前段のエージェントは、５項目程度の調査対象トピックを指定します。

* 出力形式
日本語で出力。
調査レポートは、トピックごとに客観的情報をまとめます。各トピックについて、５文以上の長さで記述すること。
'''

research_agent2 = LlmAgent(
    name='research_agent2',
    model='gemini-2.5-flash',
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント（レポート作成）',
    instruction=instruction,
)

In [6]:
instruction = '''
あなたの役割は、特定のテーマに関する気軽な読み物記事を書くことです。
記事の「テーマ」と、その内容に関連する「調査レポート」が与えられるので、
調査レポートに記載の客観的事実に基づいて、信頼性のある読み物記事を書いてください。

レビュアーによる修正ポイントが指摘された際は、直前に書いた記事を指摘に従って書き直してください。

**出力条件**
- トピックに関してある程度の基礎知識がある読者を前提として、数分で気軽に読める内容にしてください。
- 比較的カジュアルで語りかける口調の文章にします。
- 思考過程は出力せずに、最終結果だけを出力します。
- 記事タイトルは付けないで、次の構成で出力します。各セクションタイトルは、内容に合わせてアレンジしてください。
0. 導入：セクションタイトルを付けないで、この記事を読みたくなる導入を１〜２文にまとめます。
1. 概要：トピックの全体像をまとめて簡単に説明します。
2. 最新情報：特に注目したい新しい情報を取り上げます。
3. 実践：トピックに関して、読者自身がやってみるとよさそうな事を１つ紹介します。
4. まとめ

- 各セクションのタイトルはマークダウンヘッダ ## 付けます。必要に応じて箇条書きのマークダウンを使用します。
- それ以外のマークダウンによる装飾は行いません。
'''

writer_agent = LlmAgent(
    name='writer_agent',
    model='gemini-2.5-flash',
    description='特定のテーマに関する読み物記事を書くエージェント',
    instruction=instruction,
)

In [7]:
instruction = f'''
あなたの役割は、読み物記事をレビューして、記事の条件にあった内容にするための改善コメントを与える事です。

* 記事の条件
- 記事は、はじめに４０文字程度のタイトルがあること。
　今日から役立つ生活情報があって「すぐに読まなきゃ」と読者が感じるタイトルにすること。
　タイトルはマークダウンヘッダ # をつけること。
- タイトルの直後に「なぜいまこのテーマを取り上げるのか」をまとめた導入を加えて、読者にこの記事を読む動機づけを与えます。
- 各セクションのサブタイトルには、絵文字を用いて親しみやすさを出すこと。
- 読者が今日から実践できる具体例が３つ以上紹介されていること。

* 出力形式
- 日本語で出力。
- はじめに、記事の良い点を説明します。
- 次に、修正ポイントを箇条書きで出力します。
'''

review_agent = LlmAgent(
    name='review_agent',
    model='gemini-2.5-flash',
    description='読み物記事をレビューするエージェント',
    instruction=instruction,
)

## Agent Engine にデプロイ

In [47]:
from a2a.types import AgentCard
from google.adk.a2a.executor.a2a_agent_executor import A2aAgentExecutor
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.artifacts import InMemoryArtifactService
from google.adk.auth.credential_service.in_memory_credential_service import InMemoryCredentialService
from google.adk.a2a.utils.agent_card_builder import AgentCardBuilder

from vertexai.preview.reasoning_engines import A2aAgent
from vertexai.preview.reasoning_engines.templates.a2a import create_agent_card

from google.adk.sessions import VertexAiSessionService

# Wrapper class of VertexAiSessionService that's compatible with InMemorySessionService
class MyVertexAiSessionService(VertexAiSessionService):

    async def create_session(self, app_name, user_id, state={}, session_id=None):
        # Ignore session_id
        session = await super().create_session(
            app_name=app_name,
            user_id=user_id,
            state=state,
        )
        return session

    async def get_session(self, app_name, user_id, session_id):
        # Return None if session non exists.
        try:
            session = await super().get_session(
                app_name=app_name,
                user_id=user_id,
                session_id=session_id,
            )
            return session
        except:
            return None


def get_create_runner_class(agent, resource_name):
    async def create_runner():
        return Runner(
            # Use resource_name as app_name for VertexAiSessionService.
            app_name=resource_name or agent.name,
            agent=agent,
            artifact_service=InMemoryArtifactService(),
            session_service=MyVertexAiSessionService(),
            memory_service=InMemoryMemoryService(),
            credential_service=InMemoryCredentialService(),
        )
    return create_runner


def get_agent_executor_class(agent, resource_name):
    def agent_executor_builder():
        return A2aAgentExecutor(
            runner=get_create_runner_class(agent, resource_name),
        )       
    return agent_executor_builder


async def get_agent_card(agent):
    builder = AgentCardBuilder(agent=agent)
    adk_agent_card = await builder.build()
    return create_agent_card(
        agent_name=adk_agent_card.name,
        description=adk_agent_card.description,
        skills=adk_agent_card.skills
    )

ここでは、一度、ダミーでデプロイして VertexAiSessionService で必要となる Agent Engine のリソース ID を取得した後、リソース ID を指定して再デプロイしています。

In [48]:
def get_agent_resource(agent_name):
    for agent in agent_engines.list():
        if agent.display_name == agent_name:
            return agent.resource_name
    return None

In [49]:
agents = {
    'research_agent1_remote': research_agent1,
    'research_agent2_remote': research_agent2,
    'writer_agent_remote': writer_agent,
    'review_agent_remote': review_agent,
}

client = vertexai.Client(
    project=PROJECT_ID,
    location=LOCATION,
    http_options=HttpOptions(
        api_version="v1beta1", base_url=f"https://{LOCATION}-aiplatform.googleapis.com/"
    ),
)

for agent_name, agent in agents.items():    
    a2a_agent = A2aAgent(
        agent_card=await get_agent_card(agent),
        agent_executor_builder=get_agent_executor_class(agent, None)
    )
    config={
        "display_name": agent_name,
        "description": a2a_agent.agent_card.description,
        "requirements": [
            'google-adk==1.14.1',
            'google-genai==1.36.0',
            'google-cloud-aiplatform==1.113.0',
            'a2a-sdk==0.3.5'
        ],
        "http_options": {
            "base_url": f"https://{LOCATION}-aiplatform.googleapis.com",
            "api_version": "v1beta1",
        },
        "staging_bucket": f'gs://{PROJECT_ID}',
    }

    resource_name = get_agent_resource(agent_name)
    if not resource_name:
        # Deploy dummy agent to assign Agent Engine resource name
        remote_a2a_agent = client.agent_engines.create(
            agent=a2a_agent,
            config=config,
        )
    
    # Redeploy with VertexAiSessionService
    resource_name = get_agent_resource(agent_name)
    a2a_agent = A2aAgent(
        agent_card=await get_agent_card(agent),
        agent_executor_builder=get_agent_executor_class(agent, resource_name)
    )
    remote_a2a_agent = client.agent_engines.update(
        name=resource_name,
        agent=a2a_agent,
        config=config,
    )

/var/tmp/ipykernel_4435/2996848671.py:63: UserWarning: [EXPERIMENTAL] AgentCardBuilder: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  builder = AgentCardBuilder(agent=agent)
The following requirements are missing: {'pydantic', 'a2a', 'cloudpickle'}
/var/tmp/ipykernel_4435/2996848671.py:63: UserWarning: [EXPERIMENTAL] AgentCardBuilder: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  builder = AgentCardBuilder(agent=agent)
The following requirements 

## RemoteA2aAgent クラスのインスタンスを作成

In [50]:
import os
import httpx
from a2a.client import ClientConfig, ClientFactory
from a2a.types import TransportProtocol
from google.adk import Agent
from google.adk.agents.remote_a2a_agent import RemoteA2aAgent
from google.auth import default
from google.auth.transport.requests import Request


credentials, _ = default(scopes=['https://www.googleapis.com/auth/cloud-platform'])
credentials.refresh(Request())
factory = ClientFactory(
    ClientConfig(
        supported_transports=[TransportProtocol.http_json],
        use_client_preference=True,
        httpx_client=httpx.AsyncClient(
            timeout=60, # This is important.
            headers={
                "Authorization": f"Bearer {credentials.token}",
                "Content-Type": "application/json",
            }
        ),
    )
)

resource_name = get_agent_resource('research_agent1_remote')
a2a_url = f"https://{LOCATION}-aiplatform.googleapis.com/v1beta1/{resource_name}/a2a"
research_agent1_remote = RemoteA2aAgent(
    name="research_agent1",
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント（テーマ選定）',
    agent_card=f"{a2a_url}/v1/card",
    a2a_client_factory=factory,
)

resource_name = get_agent_resource('research_agent2_remote')
a2a_url = f"https://{LOCATION}-aiplatform.googleapis.com/v1beta1/{resource_name}/a2a"
research_agent2_remote = RemoteA2aAgent(
    name="research_agent2",
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント（レポート作成）',
    agent_card=f"{a2a_url}/v1/card",
    a2a_client_factory=factory,
)

resource_name = get_agent_resource('writer_agent_remote')
a2a_url = f"https://{LOCATION}-aiplatform.googleapis.com/v1beta1/{resource_name}/a2a"
writer_agent_remote = RemoteA2aAgent(
    name="writer_agent",
    description='特定のテーマに関する読み物記事を書くエージェント',
    agent_card=f"{a2a_url}/v1/card",
    a2a_client_factory=factory,
)

resource_name = get_agent_resource('review_agent_remote')
a2a_url = f"https://{LOCATION}-aiplatform.googleapis.com/v1beta1/{resource_name}/a2a"
review_agent_remote = RemoteA2aAgent(
    name="review_agent",
    description='読み物記事をレビューするエージェント',
    agent_card=f"{a2a_url}/v1/card",
    a2a_client_factory=factory,
)

/var/tmp/ipykernel_4435/2722862927.py:29: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  research_agent1_remote = RemoteA2aAgent(
/var/tmp/ipykernel_4435/2722862927.py:38: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  research_agent2_remote = RemoteA2aAgent(
/var/tmp/ipykernel_4435/2722862927.py:47: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: ADK Implementation

## Root agent の定義

In [51]:
def get_print_agent(text):
    def before_model_callback(
        callback_context: CallbackContext, llm_request: LlmRequest
    ) -> LlmResponse:
        return LlmResponse(
            content=Content(
                role='model', parts=[Part(text=text)],
            )
        )
    return LlmAgent(
        name='print_agent',
        model='gemini-2.0-flash', # not used
        description='',
        instruction = '',
        before_model_callback=before_model_callback,
    )

research_agent = SequentialAgent(
    name='research_agent',
    sub_agents=[
        get_print_agent('\n---\n## リサーチエージェントが調査レポートを作成します。\n---\n'),
        get_print_agent('\n## 調査対象のトピックを選定します。\n'),
        research_agent1_remote,
        get_print_agent('\n## 選定したトピックに基づいて、調査レポートを作成します。\n'),
        research_agent2_remote,
        get_print_agent('\n#### 調査レポートが準備できました。記事の作成に取り掛かってもよいでしょうか？\n'),
    ],
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント',
)

write_and_review_agent = SequentialAgent(
    name='write_and_review_agent',
    sub_agents=[
        get_print_agent('\n---\n## ライターエージェントが記事を執筆します。\n---\n'),
        writer_agent_remote,
        get_print_agent('\n---\n## レビューエージェントが記事をレビューします。\n---\n'),
        review_agent_remote,
       get_print_agent('\n#### レビューに基づいて記事の修正を依頼しますか？\n'),
    ],
    description='記事を作成、レビューする。',
)

In [52]:
root_agent = LlmAgent(
    name='article_generation_flow',
    model='gemini-2.0-flash',
    instruction = '''
何ができるか聞かれた場合は、以下の処理をすることをわかりやすくフレンドリーな文章にまとめて返答してください。

- ユーザーが指定したテーマの記事を作成する業務フローを実行する。
- はじめに、テーマに関する調査レポートを作成する。
- その後、ライターエージェントとレビューエージェントが協力して、編集方針に則した記事を作成する。

ユーザーが記事のテーマを指定した場合は、次のフローを実行します。

1. そのテーマの記事の作成に取り掛かる旨を伝えて、research_agent に転送して、調査レポートを依頼します。
2. ユーザー記事の作成を支持したら、write_and_review_agent に転送して、記事の作成とレビューを依頼します。
3. ユーザーが記事の修正を希望する場合は、write_and_review_agent に転送します。

**条件**
research_agent のニックネームは、リサーチエージェント
write_and_review_agent のネックネームは、ライターエージェントとレビューエージェント

''',
    sub_agents=[
        research_agent,
        write_and_review_agent,
    ],
    description='記事を作成する業務フローを実行するエージェント'
)

## 実行例

In [53]:
client = LocalApp(root_agent)
DEBUG = False

query = '''
こんにちは。何ができますか？
'''
result = await client.stream(query)

こんにちは！私は記事を作成する業務フローを実行するエージェントです。具体的には、ユーザーさんが指定したテーマに関する記事を作成します。

まず、テーマについてのリサーチエージェントが調査レポートを作成します。その後、ライターエージェントとレビューエージェントが協力して、編集方針に沿った記事を作成します。ご希望の記事テーマがあれば教えてください！



In [54]:
query = '''
「夏の節電対策」をテーマに記事を作成してください。
'''
result = await client.stream(query)

/home/jupyter/.local/lib/python3.10/site-packages/google/adk/agents/remote_a2a_agent.py:382: UserWarning: [EXPERIMENTAL] convert_genai_part_to_a2a_part: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  converted_part = self._genai_part_converter(part)


承知いたしました。「夏の節電対策」の記事作成に取り掛かります。まずはリサーチエージェントに調査レポートを依頼します。


---
## リサーチエージェントが調査レポートを作成します。
---


## 調査対象のトピックを選定します。



/home/jupyter/.local/lib/python3.10/site-packages/google/adk/a2a/converters/event_converter.py:210: UserWarning: [EXPERIMENTAL] convert_a2a_message_to_event: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  return convert_a2a_message_to_event(
/home/jupyter/.local/lib/python3.10/site-packages/google/adk/a2a/converters/event_converter.py:280: UserWarning: [EXPERIMENTAL] convert_a2a_part_to_genai_part: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  par

承知いたしました。「夏の節電対策」に関する記事作成のための調査レポート作成に向け、以下のトピックを選定します。

---

### 調査対象トピック

1.  **夏の節電対策の重要性と背景**:
    *   電気料金高騰の現状と予測
    *   夏の電力需給ひっ迫リスク
    *   地球温暖化対策としての節電の意義
    *   節電による家計へのメリット

2.  **家庭で実践できる具体的な節電方法（家電別）**:
    *   **エアコン**: 設定温度の目安、フィルター清掃、室外機周辺の整備、扇風機・サーキュレーターとの併用など
    *   **冷蔵庫**: 設定温度、開閉回数、詰め込みすぎ防止、壁との間隔など
    *   **照明**: LED化、こまめな消灯、日中の採光活用
    *   **その他家電**: テレビ、パソコン、温水洗浄便座、待機電力対策（コンセント抜き、省エネタップ）など

3.  **快適性を維持しつつ節電する工夫**:
    *   遮光カーテンやブラインドの活用による日差し対策
    *   打ち水、窓の開け方などの自然換気
    *   体のクールダウン（クールグッズ、服装、食事）
    *   熱中症予防との両立のポイント

4.  **節電をサポートする製品やサービス**:
    *   省エネ性能の高い家電への買い替えメリット
    *   スマート家電・HEMS（家庭用エネルギー管理システム）の活用
    *   電力会社の料金プランの見直し、デマンドレスポンスなど
    *   太陽光発電、蓄電池などの導入

5.  **無理なく節電を継続するための習慣と意識**:
    *   家族で取り組む節電ルール作り
    *   節電効果の可視化（スマートメーター、アプリなど）
    *   「我慢しない節電」の考え方
    *   節電とリフレッシュのバランス

---

## 選定したトピックに基づいて、調査レポートを作成します。

## 夏の節電対策に関する調査レポート

### 1. 夏の節電対策の重要性と背景

近年、電気料金は燃料費の高騰や再生可能エネルギー発電促進賦課金の影響を受け、上昇傾向にあります。特に夏場はエアコンなどの使用により電力消費が急増し、電力

In [55]:
query = '''
お願いします。
'''
result = await client.stream(query)

/home/jupyter/.local/lib/python3.10/site-packages/google/adk/agents/remote_a2a_agent.py:382: UserWarning: [EXPERIMENTAL] convert_genai_part_to_a2a_part: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  converted_part = self._genai_part_converter(part)


承知いたしました。それでは、ライターエージェントとレビューエージェントに記事の作成を依頼します。


---
## ライターエージェントが記事を執筆します。
---



/home/jupyter/.local/lib/python3.10/site-packages/google/adk/a2a/converters/event_converter.py:210: UserWarning: [EXPERIMENTAL] convert_a2a_message_to_event: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  return convert_a2a_message_to_event(
/home/jupyter/.local/lib/python3.10/site-packages/google/adk/a2a/converters/event_converter.py:280: UserWarning: [EXPERIMENTAL] convert_a2a_part_to_genai_part: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  par

今年の夏も暑くなる予感…電気代の請求書を見てため息をつく前に、今からできる夏の節電対策を一緒に見ていきましょう！

## 概要

最近、電気代が上がって家計を圧迫していると感じる方も多いのではないでしょうか。特に夏はエアコンの使用で電力消費が急増し、電力需給ひっ迫のリスクも高まります。節電は、電気代を抑えるだけでなく、社会全体の電力安定供給や地球温暖化対策にもつながる、まさに「一石三鳥」の取り組みなんです。無理なく賢く節電して、快適な夏を過ごしましょう！

## 最新情報

「節電って我慢するんでしょ？」と思われがちですが、今は便利な製品やサービスがたくさんありますよ！

*   **省エネ家電への買い替え**: 最新のエアコンや冷蔵庫は、昔のモデルと比べて格段に省エネ性能が向上しています。買い替えで初期投資はかかりますが、長期的に見れば電気代の大きな削減につながることも。
*   **スマート家電・HEMS**: スマホで家電を操作したり、家全体の電力消費をリアルタイムで見える化したりできるシステムです。無駄な電力消費を特定しやすくなるので、効率的な節電に役立ちます。
*   **料金プランの見直し**: 電力会社のプランには、夜間が安くなる「時間帯別料金」など、ライフスタイルに合わせた様々なものがあります。自分の使い方に合ったプランに変えるだけで、電気代が安くなることも！

これらを活用すれば、もっとかしこく、そして快適に節電できますよ。

## 実践

今日からすぐにできる節電術として、やっぱり主役は「エアコン」と「扇風機・サーキュレーター」の合わせ技です！

1.  **エアコンの設定温度は28℃を目安に**: 室温を下げすぎず、ちょっと高めかな？と感じるくらいが目安です。
2.  **扇風機・サーキュレーターで空気を循環**: エアコンと併用することで、冷たい空気が部屋全体に行き渡り、体感温度が下がります。これにより、エアコンの設定温度を上げても快適に過ごせます。
3.  **フィルターのお掃除はこまめに**: 2週間に一度くらいの頻度でフィルターをきれいにすると、冷房効率が約5～10%もアップするんです！ホコリが詰まっていると余計な電力を消費してしまいますからね。

この3つを意識するだけで、体感温度は変わらず、電気代はしっかりセーブできますよ。

##

In [56]:
query = '''
お願いします。
'''
result = await client.stream(query)

/home/jupyter/.local/lib/python3.10/site-packages/google/adk/agents/remote_a2a_agent.py:382: UserWarning: [EXPERIMENTAL] convert_genai_part_to_a2a_part: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  converted_part = self._genai_part_converter(part)


承知いたしました。レビュー結果を元に、ライターエージェントとレビューエージェントに記事の修正を依頼します。


---
## ライターエージェントが記事を執筆します。
---



/home/jupyter/.local/lib/python3.10/site-packages/google/adk/a2a/converters/event_converter.py:210: UserWarning: [EXPERIMENTAL] convert_a2a_message_to_event: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  return convert_a2a_message_to_event(
/home/jupyter/.local/lib/python3.10/site-packages/google/adk/a2a/converters/event_converter.py:280: UserWarning: [EXPERIMENTAL] convert_a2a_part_to_genai_part: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  par

# 【今日からできる！】快適さはそのまま！夏の電気代を賢く抑える秘訣

今年の夏も暑くなりそうですね。電気代が高騰する中、どうやって快適に過ごしながら節電するかが頭の痛い問題ではないでしょうか。でも、ご安心ください！我慢せずに、賢く電気代を抑える方法はたくさんあります。この記事を読めば、あなたの夏の電気代に対する見方がきっと変わりますよ。

## ✨ はじめに：夏の電気代、もう怖くない！

近年、電気料金は燃料価格の高騰や円安の影響で上昇傾向にあり、家計への負担は増すばかりです。特に電力消費量が増える夏は、熱中症予防のためにエアコンを適切に使うことが必須。しかし、「快適さ」と「節電」を両立させるのは難しいと感じるかもしれませんね。でも、ちょっとした工夫で、この二つは無理なく両立できます。夏の節電は、家計に優しく、地球環境にも貢献し、さらには快適な生活を維持するという、まさに一石三鳥の素晴らしい取り組みなんです。

## 💡 知って得する！最新の節電トレンド

ただ闇雲に電気を消すだけが節電ではありません。最近では、もっとスマートで効率的な節電方法が登場しています。

*   **省エネ家電への買い替え**：冷蔵庫やエアコンなどの主要家電を最新の省エネモデルに買い替えることで、目に見えて電気代が安くなることがあります。初期費用はかかりますが、長期的に見ればお得になるケースが多いですよ。
*   **スマート家電・HEMS（ヘムス）の活用**：スマートフォンで家電を遠隔操作したり、家庭内のエネルギー使用量を「見える化」したりするシステムです。無駄な電力消費をリアルタイムで把握できるので、効率的な節電につながります。
*   **料金プランの見直し**：電力会社によっては、時間帯によって料金が変わるプランや、再生可能エネルギーを利用するプランなど、様々な選択肢があります。ご自身のライフスタイルに合ったプランを選ぶだけで、電気代が安くなる可能性も。

## 🚀 今日からできる！賢い節電テクニック

すぐに実践できる、具体的な節電術をいくつかご紹介します。これらを組み合わせることで、大きな効果が期待できますよ。

*   **エアコンの賢い使い方**
    *   **設定温度は無理なく高めに**: 環境省が推奨する室温の目安は28℃ですが、無理のない範囲で調整しましょう。

In [57]:
query = '''
これで完成です。全文を表示してください。
'''
result = await client.stream(query)

# 【今日からできる！】快適さはそのまま！夏の電気代を賢く抑える秘訣

今年の夏も暑くなりそうですね。電気代が高騰する中、どうやって快適に過ごしながら節電するかが頭の痛い問題ではないでしょうか。でも、ご安心ください！我慢せずに、賢く電気代を抑える方法はたくさんあります。この記事を読めば、あなたの夏の電気代に対する見方がきっと変わりますよ。

## ✨ はじめに：夏の電気代、もう怖くない！

近年、電気料金は燃料価格の高騰や円安の影響で上昇傾向にあり、家計への負担は増すばかりです。特に電力消費量が増える夏は、熱中症予防のためにエアコンを適切に使うことが必須。しかし、「快適さ」と「節電」を両立させるのは難しいと感じるかもしれませんね。でも、ちょっとした工夫で、この二つは無理なく両立できます。夏の節電は、家計に優しく、地球環境にも貢献し、さらには快適な生活を維持するという、まさに一石三鳥の素晴らしい取り組みなんです。

## 💡 知って得する！最新の節電トレンド

ただ闇雲に電気を消すだけが節電ではありません。最近では、もっとスマートで効率的な節電方法が登場しています。

*   **省エネ家電への買い替え**：冷蔵庫やエアコンなどの主要家電を最新の省エネモデルに買い替えることで、目に見えて電気代が安くなることがあります。初期費用はかかりますが、長期的に見ればお得になるケースが多いですよ。
*   **スマート家電・HEMS（ヘムス）の活用**：スマートフォンで家電を遠隔操作したり、家庭内のエネルギー使用量を「見える化」したりするシステムです。無駄な電力消費をリアルタイムで把握できるので、効率的な節電につながります。
*   **料金プランの見直し**：電力会社によっては、時間帯によって料金が変わるプランや、再生可能エネルギーを利用するプランなど、様々な選択肢があります。ご自身のライフスタイルに合ったプランを選ぶだけで、電気代が安くなる可能性も。

## 🚀 今日からできる！賢い節電テクニック

すぐに実践できる、具体的な節電術をいくつかご紹介します。これらを組み合わせることで、大きな効果が期待できますよ。

*   **エアコンの賢い使い方**
    *   **設定温度は無理なく高めに**: 環境省が推奨する室温の目安は28℃ですが、無理のない範囲で調整しましょう。